In [2]:
import pymupdf
import logging
import io
import uuid
import time
import json
import requests
from pypdf import PdfWriter
from collections import defaultdict
from fastapi import APIRouter, HTTPException
# from models.render import DocumentRendererResponse

In [ ]:
logger = logging.getLogger(__name__)
doc_url="/home/ubuntu/Desktop/OmniPDF/sample-files/sample-tables.pdf"
json_url="/home/ubuntu/Desktop/OmniPDF/sample-files/table_output.json"

# response = requests.get(json_url)
# response.raise_for_status()
# json_data = response.json()

with open(json_url, "r", encoding="utf-8") as f:
    json_data = json.load(f)



trans_text_data = defaultdict(list)

texts = json_data.get("docling", {}).get("texts", [])

for text_item in texts:
    translated = text_item.get("translated_text", "")

    for prov in text_item.get("prov", []):
        page_no = prov.get("page_no")
        bbox = prov.get("bbox")

        if page_no is not None and bbox:
            trans_text_data[page_no].append({
                "translated_text": translated,
                "bbox": bbox
            })


tables = json_data.get("docling", {}).get("tables", [])

for table in tables:
    table_cells = table.get("data", {}).get("table_cells", [])
    page_no = table.get("prov", [])[0].get("page_no")

    for cell in table_cells:
        translated = cell.get("translated_text")
        bbox = cell.get("bbox")

        y0 = bbox["b"]
        y1 = bbox["t"]

        if page_no is not None and bbox:
            trans_text_data[page_no].append({
                "translated_text": translated,
                "bbox": bbox
            })


data = dict(trans_text_data)

# pdf_response = requests.get(doc_url)
# pdf_response.raise_for_status()
doc = pymupdf.open(doc_url, filetype="pdf")

for page in doc:
    logger.info(f"{page.number}\n") 
    blocks = page.get_text("blocks") 
    for block in blocks:
        rect = block[:4] 

        page.add_redact_annot(rect)

    page.apply_redactions()
    page.clean_contents()
    data_lst = data[page.number + 1]
    for trans_data in data_lst:
        trans_text = trans_data["translated_text"]
        bbox = trans_data['bbox']

        y0 = (bbox["t"])
        y1 = (bbox["b"])

        top = max(y0, y1)
        bottom = min(y0, y1)

        coords = (bbox["l"], top, bbox["r"], bottom)
        
        print(f"Trans Text: {trans_text}")
        print(f"Coords: {coords}")

        # logger.info(f"Text: {trans_text}")
        # logger.info(f"Bbox: {coords}")
        try:
            page.insert_htmlbox(coords, trans_text)
        except Exception as e:
            # logger.error("Error inserting HTML box:")
            # logger.error(f"Text: {trans_text}")
            # logger.error(f"Original BBox: {bbox}")
            # logger.error(f"Converted Coords: {coords}")
            # logger.error(f"Page size: {page.rect}")
            # logger.exception(e)  # full traceback
            raise e  # optionally re-raise for FastAPI to return 500

        


# logger.info(f"Time to render document: {time.time() - start_time}")

# doc_id = str(uuid.uuid4())
# key = f"{doc_id}/rendered.pdf"

# try:
#     success = upload_fileobj(
#         buffer, key, content_type="application/pdf"
#     )
#     if not success:
#         raise HTTPException(status_code=500, detail="Failed to upload file to S3")

#     presigned_url = generate_presigned_url(key)

# except Exception as e:
#     logger.error(f"Unexpected error during upload: {e}", exc_info=True)
#     raise HTTPException(status_code=500, detail="Internal server error")
    
#     return(DocumentRendererResponse(doc_id=doc_id,
#                                     filename=key,
#                                     download_url=presigned_url,
#                                     )
#                                 )

Trans Text: 设计并构建可访问的PDF表格 示例表格
Coords: (90, 702.425, 344.845, 637.028)


OverflowError: int too large to convert to float

In [38]:
save_path = "/home/ubuntu/Desktop/OmniPDF/sample-files/test.pdf"

In [39]:
doc.subset_fonts()
doc.save(save_path, garbage=4, deflate=True, clean=True)
# garbage=3, deflate=True, clean=True

In [10]:
buffer = io.BytesIO()
doc.ez_save(buffer)
buffer.seek(0)  # Reset buffer position

0

In [26]:
writer = PdfWriter(clone_from="/home/ubuntu/Desktop/OmniPDF/pdf_renderer_service/routers/test.pdf")

for page in writer.pages:
    page.compress_content_streams()
with open("/home/ubuntu/Desktop/OmniPDF/pdf_renderer_service/routers/test.pdf", "wb") as f:
    writer.write(f)